In [218]:
from sklearn import *
import numpy as np


In [219]:
import pandas as pd

In [220]:
trainData = pd.read_csv("train.csv")

In [227]:
trainData = trainData.drop(["Ticket","Name","Cabin","PassengerId"],axis=1)

In [221]:
trainData["Sex"]=trainData["Sex"].map({"male":0,"female":1})

In [222]:
#Embarked 转化为哑变量
trainData = trainData.join(pd.get_dummies(trainData["Embarked"]).ix[:,1:])
trainData = trainData.drop(["Embarked"],axis=1)


# Fill NaN


In [223]:
#计算不同称呼人士的平均年龄
countMr = 0;countMiss = 0;countMrs = 0;countMaster = 0;countAll = 0
meanOfMr = 0;meanOfMiss = 0;meanOfMrs = 0;meanOfMaster = 0;meanOfAll = 0
for index in range(len(trainData)):
    if(pd.isnull(trainData["Age"][index])):
        continue
    if "Mr." in trainData["Name"][index]:
        countMr += 1
        meanOfMr += trainData["Age"][index]
    elif "Miss." in trainData["Name"][index]:
        countMiss += 1
        meanOfMiss += trainData["Age"][index]
        
    elif "Mrs." in trainData["Name"][index]:
        countMrs += 1
        meanOfMrs += trainData["Age"][index]
    elif "Master." in trainData["Name"][index]:
        countMaster += 1
        meanOfMaster += trainData["Age"][index]
    meanOfAll += trainData["Age"][index]
    countAll += 1
meanOfMr /= float(countMr)
meanOfMrs /= float(countMrs)
meanOfMaster /= float(countMaster)
meanOfMiss /= float(countMiss)
meanOfAll /= float(countAll)

In [224]:
pd.Series({"meanOfAll":meanOfAll,"meanOfMaster":meanOfMaster,"meanOfMiss":meanOfMiss,"meanOfMrs":meanOfMrs,"meanOfMr":meanOfMr})

meanOfAll       29.699118
meanOfMaster     4.574167
meanOfMiss      21.773973
meanOfMr        32.368090
meanOfMrs       35.898148
dtype: float64

In [225]:
for index in range(len(trainData)):
    if(pd.isnull(trainData["Age"][index])):
        if "Mr." in trainData["Name"][index]:
            trainData["Age"][index] = meanOfMr
        elif "Miss." in trainData["Name"][index]:
            trainData["Age"][index] = meanOfMiss  
        elif "Mrs." in trainData["Name"][index]:
            trainData["Age"][index] = meanOfMrs
        elif "Master." in trainData["Name"][index]:
            trainData["Age"][index] = meanOfMaster
        else:
            trainData["Age"][index] = meanOfAll

H:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
H:\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
H:\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
H:\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: htt

In [226]:
trainData.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Q,S
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,0,0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0,1
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0,1


In [229]:
from sklearn import preprocessing as prep

In [230]:

#scale data with minmax_scale
trainData = pd.DataFrame(prep.minmax_scale(trainData),columns=trainData.columns)

In [231]:
trainData.tail()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Q,S
886,0.0,0.5,0.0,0.334004,0.000,0.000000,0.025374,0.0,1.0
887,1.0,0.0,1.0,0.233476,0.000,0.000000,0.058556,0.0,1.0
888,0.0,1.0,1.0,0.268333,0.125,0.333333,0.045771,0.0,1.0
889,1.0,0.0,0.0,0.321438,0.000,0.000000,0.058556,0.0,0.0
890,0.0,1.0,0.0,0.396833,0.000,0.000000,0.015127,1.0,0.0


# selectFeatures


In [235]:
#Relief method 
#计算相关统计量

def getDistance(trainData,i,index):
    res = 0.0
    for feature in trainData.columns[1:]:
        res += (trainData[feature][i] - trainData[feature][index])**2
    return res


def getNearest(trainData,i,label):
    minDis = 99.0
    res = 0
    for index in range(len(trainData)):
        if(index == i):
            continue
        survived = trainData["Survived"][index]
        if survived == label:
            distance = getDistance(trainData,i,index)
            if distance < minDis:
                res = index
                minDis = distance
    return res
def selectFeatures(trainData):
    features = trainData.columns[1:]
    delta = {}
    for feature in features:
        print ("feature : %s",feature)
        delta[feature] = 0.0
        for index in range(len(trainData)):
            print ("index:",index)
            survived = trainData["Survived"][index]
            nearHitIndex = getNearest(trainData,index,survived if survived else 0.0)
            nearMissIndex = getNearest(trainData,index,0.0 if survived else survived)
            delta[feature] += -(trainData[feature][nearHitIndex] - trainData[feature][index]) **2 + \
                                (trainData[feature][nearMissIndex] - trainData[feature][index]) **2
    return delta

In [198]:
trainData.columns[1:]

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Q', 'S'], dtype='object')

In [ ]:
selectFeatures(trainData)

feature : %s Pclass
index: 0
index: 1
index: 2
index: 3
index: 4
index: 5
index: 6
index: 7
index: 8
index: 9
index: 10
index: 11
index: 12
index: 13
index: 14
index: 15
index: 16
index: 17
index: 18
index: 19
index: 20
index: 21
index: 22
index: 23
index: 24
index: 25
index: 26
index: 27
index: 28
index: 29
index: 30
index: 31
index: 32
index: 33
index: 34
index: 35
index: 36
index: 37
index: 38
index: 39
index: 40
index: 41
index: 42
index: 43
index: 44
index: 45
index: 46
index: 47
index: 48
index: 49
index: 50
index: 51
index: 52
index: 53
index: 54
index: 55
index: 56
index: 57
index: 58
index: 59
index: 60
index: 61
index: 62
index: 63
index: 64
index: 65
index: 66
index: 67
index: 68
index: 69
index: 70
index: 71
index: 72
index: 73
index: 74
index: 75
index: 76
index: 77
index: 78
index: 79
index: 80
index: 81
index: 82
index: 83
index: 84
index: 85
index: 86
index: 87
index: 88
index: 89


<function numpy.core.fromnumeric.amax>